<a href="https://colab.research.google.com/github/williamzhao01123/crypto-lab/blob/master/Deciphering_Bitcoin_Blockchain_Data_by_Cohort_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is the codebook for the paper *Deciphering Bitcoin Blockchain Data by Cohort Analysis*, authored by Yulin Liu, Luyao Zhang, and Yinhong Zhao. The notebook provides all the codes we used to derive the results presented in the paper.

# Part I: Data Querying

## Preliminaries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import decimal
from datetime import datetime, date, timedelta, timezone

In [ ]:
#Connect to Google Cloud
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Authenticated')

In [ ]:
#Connect to Google BigQuery
PROJECT_ID = 'crypto-291811'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location='US')
dataset_ref = client.dataset('UTXO', project=PROJECT_ID)
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
  print(table.table_id)

## Creating a Table for variables of interest

In [ ]:
#Create joint_all

table_id_inputs = "crypto-291811.UTXO.joint_all"
job_config = bigquery.QueryJobConfig(destination=table_id_inputs)

sql = """
  SELECT
    (outputs.value/POW(10,8)) AS UTXO,  
    outputs.block_timestamp,
    inputs.block_timestamp AS spent_block_timestamp,
    #FORMAT_TIMESTAMP("%Y-%m-%d", block_timestamp) AS block_date,
    #FORMAT_TIMESTAMP("%Y-%m-%d", spent_block_timestamp) AS spent_block_date,
  FROM 
    `bigquery-public-data.crypto_bitcoin.outputs` AS outputs
  LEFT JOIN 
    `bigquery-public-data.crypto_bitcoin.inputs` AS inputs
  ON outputs.transaction_hash=inputs.spent_transaction_hash  
  AND outputs.index = inputs.spent_output_index
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.


print("Query results loaded to the table {}".format(table_id_inputs))

## Create partitioned tables

In [ ]:
#Partition Table by born date for data after 2012

job_config = bigquery.QueryJobConfig()
sql = """
  CREATE TABLE
    `crypto-291811.UTXO.joint_all_partitionedbyborn12`
  PARTITION BY
    DATE(block_timestamp) AS
  SELECT
    *
  FROM
    `crypto-291811.UTXO.joint_all`
  WHERE
    block_timestamp > TIMESTAMP('2012-01-01 00:00:00+00')
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id_inputs))

In [ ]:
#Partition by death date for data after 2012

job_config = bigquery.QueryJobConfig()
sql = """
  CREATE TABLE
    `crypto-291811.UTXO.joint_all_partitionedbydeath12`
  PARTITION BY
    DATE(spent_block_timestamp) AS
  SELECT
    *
  FROM
    `crypto-291811.UTXO.joint_all`
  WHERE
    (spent_block_timestamp > TIMESTAMP('2012-01-01 00:00:00+00')
    OR 
    spent_block_timestamp IS NULL)

"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id_inputs))

# Part II: Data Processing

## Define Functions

In [ ]:
def cal(x):
    t=np.sign(x-0.999)+np.sign(x-29.999)+np.sign(x-90.999)+np.sign(x-181.999)+np.sign(x-364.999)+np.sign(x-365*2+0.001)+np.sign(x-365*3+0.001)+np.sign(x-365*4+0.001)+np.sign(x-365*5+0.001)+np.sign(x-365*10+0.001)+1
    return t

def Task1_born(data):
    newborn = data['UTXO'].sum()
    return(newborn)

#Partitioning By Death Date
def Task1_dead(data):
    dead = data['UTXO'].sum()
    return(dead)

def Task2(data):
    #data['Life_Length'] = data['spent_block_timestamp']- data['block_timestamp']
    #data['Life_Length'] = data['Life_Length'].map(lambda x:x.days).apply(float)
    sumUTXO = data['UTXO'].sum()
    sumLength = (data['UTXO']*data['Life_Length']).sum()
    if sumUTXO == 0:
        WALE = 0.0
    else:
        WALE = sumLength/sumUTXO
    return(WALE)
def Task3(data):
    data['Life_Length'] = data['spent_block_timestamp']- data['block_timestamp']
    data['Life_Length'] = data['Life_Length'].map(lambda x:x.days).apply(float)
    data['categorical'] = cal(data['Life_Length'])
    categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
    result=pd.DataFrame(np.zeros((1, 11)), columns=categories)
    for i in categories:  
        result.loc[:,i] = data[data['categorical']==i]['UTXO'].sum()  
    return result

def Task4(data, date):  
    categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
    result=pd.DataFrame(np.zeros((1, 11)), columns=categories)
    if len(data)!= 0:
      data['Age'] = data['block_timestamp'].apply(lambda x: (working_date-x).days)
      data['categorical'] = cal(data['Age'])
      for i in categories: 
        result.loc[:,i] = data[data['categorical']==i]['UTXO'].sum()
    return result

## Defining the Processing Programs

In [ ]:
def STXOprogram(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Result=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Result['date'] = duration

  for i in range(0, days):
    start_date = start + timedelta(days=i)
    end_date = start_date + timedelta(days=1)
    
  #Partitioning by Dead Date
    query2 = """
          SELECT 
            *
          FROM 
            `crypto-291811.UTXO.joint_all_partitionedbydeath12`
          WHERE
            spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
          AND 
            spent_block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
    query_job2 = client.query(query2)
    # Make an API request  to run the query and return a pandas DataFrame
    data2 = query_job2.to_dataframe()
    data2['UTXO'] = (data2['value']/10**8).apply(float)
    data2 = data2.drop(['value'], axis = 1)  
    
    #Work on Task3
    Result.iloc[i,0:10]=list(Task3(data2).iloc[0])
    
  #Partitioning by Born Date
    query1 = """
          SELECT 
            *
          FROM 
            `crypto-291811.UTXO.joint_all_partitionedbyborn12`
          WHERE
            block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
          AND 
            block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
    query_job1 = client.query(query1)
    # Make an API request  to run the query and return a pandas DataFrame
    data1 = query_job1.to_dataframe()
    data1['UTXO'] = (data1['value']/10**8).apply(float)
    data1 = data1.drop(['value'], axis = 1)
    
    #Work on Task 1 and Task 2
    Result.loc[i,'newborn'] = Task1_born(data1)
    Result.loc[i,'dead'] = Task1_dead(data2)
    Result.loc[i,'WALE'] = Task2(data2)
  return Result

In [ ]:
def UTXOprogram(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9]
  Dist_Alive=pd.DataFrame(np.zeros((days, 10)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all_pdeathcborn`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
      AND 
        (spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
        OR 
        spent_block_timestamp IS NULL)
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[((data.block_timestamp<working_date) & ((pd.isna(data.spent_block_timestamp) | (data.spent_block_timestamp>=working_date))))].copy()
    Dist_Alive.iloc[j,0:10] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

## Running the Processing Programs

In [ ]:
for year in range(2012, 2021):
  start = date(year,1,1)
  end = date(year,12,31)
  STXOresult = STXOprogram(start, end)
  address = '/content/drive/My Drive/ResultSTXO' + str(year) + '.csv'
  STXOresult.to_csv(address)

In [ ]:
for year in range(2012, 2021):
  start = date(year,1,1)
  end = date(year,12,31)
  UTXOresult = UTXOprogram(start, end)
  address = '/content/drive/My Drive/ResultUTXO' + str(year) + '.csv'
  STXOresult.to_csv(address)

Note: this is how we acquire results for all data from 2012 to 2020. In practice, you may adjust the start and end range to acquire results for any time period you want.

## 2009-2011 Data

In [ ]:
#STXO
end_date = date(2012,1,1)
query1 = """
          SELECT 
            *
          FROM 
            `crypto-291811.UTXO.joint_all`
          WHERE
            block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
            
query_job1 = client.query(query1)

    # Make an API request  to run the query and return a pandas DataFrame
data = query_job1.to_dataframe()
data['UTXO'] = (data['value']/10**8).apply(float)
data = data.drop(['value'], axis = 1)

start = date(2009,1,3)
end = date(2011,12,31)
duration=pd.date_range(start=start, end=end)
days = np.size(duration)
categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9]
Result=pd.DataFrame(np.zeros((days, 10)), columns=categories)
Result['date'] = duration

for i in range(0, days):
    working_start_date = pd.to_datetime(start + timedelta(days=i),utc=True)
    working_end_date = pd.to_datetime(start +timedelta(days=1)+timedelta(days=i),utc=True)
    working_data2 = data[(data.spent_block_timestamp >= working_start_date) & (data.spent_block_timestamp < working_end_date)].copy()
    Result.iloc[i,0:10]=list(Task3(working_data2).iloc[0])
    working_data1 = data[(data.block_timestamp >= working_start_date) & (data.block_timestamp < working_end_date)].copy()
    Result.loc[i,'newborn'] = Task1_born(working_data1)
    Result.loc[i,'dead'] = Task1_dead(working_data2)
    Result.loc[i,'WALE'] = Task2(working_data2)
Result.to_csv('/content/drive/My Drive/ResultSTXO09-11.csv')

In [ ]:
start = date(2009,1,3)
end = date(2011,12,31)
duration=pd.date_range(start=start, end=end)
days = np.size(duration)
categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9]
Dist_Alive=pd.DataFrame(np.zeros((days, 10)), columns=categories)
Dist_Alive['date'] = duration

start_date=start+timedelta(days=1) 
end_date =end+timedelta(days=1) 
### note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
#must be from joint_all
query = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
     """
query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
data = query_job.to_dataframe()
data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')

for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[((data.block_timestamp<working_date) & ((pd.isna(data.spent_block_timestamp) | (data.spent_block_timestamp>=working_date))))].copy()
    Dist_Alive.iloc[j,0:10] = list(Task4(working_data, working_date).iloc[0])

Dist_Alive.to_csv('/content/drive/My Drive/UTXO/DistAlive09-11.csv')

## Merge the Results to a Final Time Series

In [ ]:
#Assuming that all necessary data files are ready in Google Drive
start = 2012
end = 2020
currency = "Bitcoin"

Task123 = pd.read_csv("ResultSTXO09-11.csv")
Dist_Alive = pd.read_csv("ResultUTXO09-11.csv")

for i in range(start, end+1):
  file1 = pd.read_csv(currency + "Result" + str(i) + "Task123.csv")
  file2 = pd.read_csv(currency + "DistAlive" + str(i) + ".csv")
  Task123 = Task123.append(file1)
  Dist_Alive = Dist_Alive.append(file2)

Task123 = Task123.reset_index(drop=True).drop(['Unnamed: 0'], axis = 1)
Task123 = Task123[['date', 'newborn', 'dead', 'WALE', '-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11']]
Dist_Alive = Dist_Alive.reset_index(drop=True).drop(['Unnamed: 0'], axis = 1)
Dist_Alive = Dist_Alive[['date', '-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11']]

name1 = '/content/drive/My Drive/UTXO/' + currency + 'Task123.csv'
name2 = '/content/drive/My Drive/UTXO/' + currency + 'Distalive.csv'
Task123.to_csv(name1)
Dist_Alive.to_csv(name2)

# Part III: Data Visualization

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import decimal
import plotly.graph_objects as go
from plotly.offline import iplot
import plotly.express as px
import plotly.offline as py     
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Importing drive method from colab for accessing google drive
from google.colab import drive
# Mounting drive
drive.mount('/content/drive')

## Define the input variables 

In [ ]:
URL ="/content/drive/My Drive/UTXO/Data/Finals/" #the folder that contains all the data files
currency = "Bitcoin" #the name of the altcoin
name1 = "Task123.csv"
name2 = "Distalive.csv"
name3 = "prices.csv"

Note: The files must be compiled properly in the google drive and named correctly. The prices data is exogenous, which is downloaded at coinmetrics.

## Defining Class and Functions

In [ ]:
class Altcoin:
  def __init__(self, URL, currency):
    self.URL =URL
    self.currency =currency
  def data(self, name1, name2, name3):         #import the data
    data1 = self.URL+self.currency + name1
    data2 = self.URL+self.currency + name2
    data3 = self.URL+self.currency + name3
    result = pd.read_csv(data1,index_col='Unnamed: 0')
    Dist_Alive = pd.read_csv(data2,index_col='Unnamed: 0')
    price = pd.read_csv(data3)
    categories=['-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11']
    price["Date"] = pd.to_datetime(price["date"], format='%Y-%m-%d')
    Result = result
    Result[categories] = Result[categories].cumsum(axis=0)
    Result['net_new'] = Result['newborn'] - Result['dead']
    Result['UTXO_Cum'] = Result['net_new'].cumsum()
    Result['dead_cum'] = Result['dead'].cumsum()
    Result['lifelength_cum'] = (Result['dead'].mul(Result['WALE'])).cumsum()
    Result['WALE_cum'] = Result['lifelength_cum']/Result['dead_cum']
    result = pd.read_csv(data1,index_col='Unnamed: 0')
    return [Result, Dist_Alive, price, result]
  def Dist(self):
    import plotly.offline as py     
    import plotly.graph_objects as go
    trace0 = go.Scatter(x = Result["date"], y = Result.UTXO_Cum, hoverinfo='x+y', mode='lines', stackgroup='one', groupnorm='percent', name = 'Accumulated UTXO in '+self.currency)
    trace1 = go.Scatter(x = Result["date"], y = Result.dead_cum, hoverinfo='x+y', mode='lines', stackgroup='one', name = 'Accumulated STXO in '+self.currency)
    layout = go.Layout(title="Distribution of UTXO and Accumulated STXO in "+currency, xaxis = dict(title="date"), yaxis = dict(title ="Percentage") ) 
    data = [trace0, trace1]
    fig1 = go.Figure(layout = layout, data = data) 
    fig1.update_yaxes(type="log")
    py.iplot(fig1)
    return fig1
  def Dist_UTXO(self):
    trace0 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["-9"], hoverinfo='x+y', mode='lines', stackgroup='one', groupnorm='percent', name = '< 1d', line=dict(color="#f0f921"))
    trace1 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["-7"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1d ~ 1m', line=dict(color="#fdca26"))
    trace2 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["-5"]+Dist_Alive["-3"]+Dist_Alive["-1"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1m ~ 1y', line=dict(color="#fb9f3a"))
    trace3 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["1"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1y ~ 2y', line=dict(color="#d8576b"))
    trace4 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["3"]+Dist_Alive["5"]+Dist_Alive["7"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '2y ~ 5y', line=dict(color="#9c179e"))
    trace5 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["9"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '5y ~ 10y', line=dict(color="#46039f"))
    trace6 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["11"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '> 10y', line=dict(color="#0d0887"))
    fig2 = make_subplots(specs=[[{"secondary_y": True}]])
    fig2.add_trace(trace6)
    fig2.add_trace(trace5)
    fig2.add_trace(trace4)
    fig2.add_trace(trace3)
    fig2.add_trace(trace2)
    fig2.add_trace(trace1)
    fig2.add_trace(trace0)
    fig2.add_trace(go.Scatter(x=price['Date'], y=price['PriceUSD'], mode='lines', name='Price', line=dict(color="black")), secondary_y=True)
    fig2.update_layout(title="Age Distribution of UTXOs and Prices for "+self.currency, xaxis = dict(title="date"), yaxis = dict(title ="Percentage") ) 
    fig2.update_yaxes(type="log", title_text ="Price (USD)", secondary_y=True)
    fig2.update_yaxes(tickvals=[0.1, 1, 10, 100, 1000, 10000], secondary_y=True)
    fig2.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=0.95
    ))
    py.iplot(fig2)
    return fig2

  def Dist_UTXO_num(self):
    trace0 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["-9"], name = '< 1d', line=dict(color="#f0f921"))
    trace1 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["-7"], name = '1d ~ 1m', line=dict(color="#fb9f3a"))
    trace2 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["-5"]+Dist_Alive["-3"]+Dist_Alive["-1"], name = '1m ~ 1y', line=dict(color="#d8576b"))
    trace3 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["1"], name = '1y ~ 2y', line=dict(color="#9c179e"))
    trace4 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["3"]+Dist_Alive["5"]+Dist_Alive["7"], name = '2y ~ 5y', line=dict(color="#7201a8"))
    trace5 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["9"], name = '5y~10y', line=dict(color="#46039f"))
    trace6 = go.Scatter(x = Dist_Alive["date"], y = Dist_Alive["11"], name = '>10y', line=dict(color="#0d0887"))
    layout = go.Layout(title= "UTXOs by Age in "+self.currency, xaxis = dict(title="date"), yaxis = dict(title ="UTXOs in "+self.currency) ) 
    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6]
    fig3 = go.Figure(layout = layout, data = data) 
    fig3.update_yaxes(type="log")
    fig3.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    py.iplot(fig3)
    return fig3
  def Dist_STXO(self):
    trace0 = go.Scatter(x = Result["date"], y = Result["-9"], hoverinfo='x+y', mode='lines', stackgroup='one', groupnorm='percent', name = '< 1d', line=dict(color="#f0f921"))
    trace1 = go.Scatter(x = Result["date"], y = Result["-7"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1d ~ 1m', line=dict(color="#fb9f3a"))
    trace2 = go.Scatter(x = Result["date"], y = Result["-5"]+Result["-3"]+Result["-1"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1m ~ 1y', line=dict(color="#d8576b"))
    trace3 = go.Scatter(x = Result["date"], y = Result["1"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1y ~ 2y', line=dict(color="#9c179e"))
    trace4 = go.Scatter(x = Result["date"], y = Result["3"]+Result["5"]+Result["7"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '2y ~ 5y', line=dict(color="#7201a8"))
    trace5 = go.Scatter(x = Result["date"], y = Result["9"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '5y ~ 10y', line=dict(color="#46039f"))
    trace6 = go.Scatter(x = Result["date"], y = Result["11"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '> 10y', line=dict(color="#0d0887"))
    layout = go.Layout(title="Lifespan Distribution of Accumulated STXOs for "+self.currency, xaxis = dict(title="Date"), yaxis = dict(title ="Percentage") ) 
    data = [trace6, trace5, trace4, trace3, trace2, trace1, trace0]
    fig4 = go.Figure(layout = layout, data = data) 
    fig4.update_yaxes(type="log")
    fig4.update_yaxes(tickvals=[0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100])
    fig4.update_yaxes(range=[-1.2, 2])
    fig4.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    py.iplot(fig4)
    return fig4

  def Dist_STXO_num(self):
    trace0 = go.Scatter(x = Result["date"], y = Result["-9"], name = '< 1d')
    trace1 = go.Scatter(x = Result["date"], y = Result["-7"], name = '1d ~ 1m')
    trace2 = go.Scatter(x = Result["date"], y = Result["-5"]+Result["-3"]+Result["-1"], name = '1m ~ 1y')
    trace3 = go.Scatter(x = Result["date"], y = Result["1"], name = '1y ~ 2y')
    trace4 = go.Scatter(x = Result["date"], y = Result["3"]+Result["5"]+Result["7"], name = '2y ~ 5y')
    trace5 = go.Scatter(x = Result["date"], y = Result["9"], name = '5y ~ 10y')
    trace6 = go.Scatter(x = Result["date"], y = Result["11"], name = '> 10y')

    layout = go.Layout(title="Accumulated STXOs by Lifespan in "+self.currency, xaxis = dict(title="date"), yaxis = dict(title ="STXO") ) 
    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6]
    fig5 = go.Figure(layout = layout, data = data) 
    fig5.update_yaxes(type="log")
    fig5.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    py.iplot(fig5)
    return fig5
  def Dist_daiy_STXO(self):
    trace0 = go.Scatter(x = result["date"], y = result["-9"], hoverinfo='x+y', mode='lines', stackgroup='one', groupnorm='percent', name = '< 1d', line=dict(color="#f0f921"))
    trace1 = go.Scatter(x = result["date"], y = result["-7"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1d ~ 1m', line=dict(color="#fb9f3a"))
    trace2 = go.Scatter(x = result["date"], y = result["-5"]+result["-3"]+result["-1"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1m ~ 1y', line=dict(color="#d8576b"))
    trace3 = go.Scatter(x = result["date"], y = result["1"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '1y ~ 2y', line=dict(color="#9c179e"))
    trace4 = go.Scatter(x = result["date"], y = result["3"]+result["5"]+result["7"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '2y ~ 5y', line=dict(color="#7201a8"))
    trace5 = go.Scatter(x = result["date"], y = result["9"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '5y ~ 10y', line=dict(color="#46039f"))
    trace6 = go.Scatter(x = result["date"], y = result["11"], hoverinfo='x+y', mode='lines', stackgroup='one', name = '> 10y', line=dict(color="#0d0887"))
    layout = go.Layout(title="Lifespan Distribution of the Daily STXOs in "+self.currency, xaxis = dict(title="date"), yaxis = dict(title ="Percentage") ) 
    data = [trace6, trace5, trace4, trace3, trace2, trace1, trace0]
    fig6 = go.Figure(layout = layout, data = data) 
    fig6.update_yaxes(type="log")
    fig6.update_yaxes(tickvals=[0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100])
    fig6.update_yaxes(range=[-1.5, 2])
    fig6.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    py.iplot(fig6)
    return fig6
  def WAL(self):
    fig7 = px.line(Result, x = 'date', y = 'WALE_cum', title = 'Weighted Average Lifespan of Accumulated STXO for '+self.currency)
    fig7.update_layout(xaxis_title='Date', yaxis_title='Days')
    py.iplot(fig7)
    return fig7
  def Supply(self):
    import plotly.offline as py     
    import plotly.graph_objects as go
    fig8 = go.Figure()
    fig8.add_trace(go.Scatter(x=Result.date, y=Result.net_new,
                  mode='markers', name='block reward', marker=dict(size=4)))
    fig8.add_trace(go.Scatter(x=Result.date, y=Result.UTXO_Cum, yaxis="y2",
                  mode='lines', name='circulating supply', line=dict(color="red")))
    fig8.add_trace(go.Scatter(x=Result.date, y=Result.dead_cum, yaxis="y3",
                  mode='lines', name='total spent', line=dict(color="black")))
    fig8.update_yaxes(range=[0, 16000])
    fig8.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    fig8.update_layout(title='Block rewards, circulating supply (UTXO), and total spent (accumulated STXO) of  ' + self.currency,
              xaxis_title='Date',
              xaxis=dict(domain=[0,0.9]),
              yaxis1=dict(title="", tickfont=dict(color="blue")),
              yaxis2=dict(
                title="",
                titlefont=dict(color="red"),
                tickfont=dict(color="red"),
              anchor="x",
              overlaying="y",
              side="right"),
              yaxis3=dict(
                title="",
                titlefont=dict(color="black"),
                tickfont=dict(color="black"),
              anchor="free",
              overlaying="y",
              side="right",
              position=0.95)
    )
    py.iplot(fig8)
    return fig8
  def Velocity(self, Result, days):
    for i in range(days,len(Result)):
      Result['STXO_diff'] = Result['dead_cum'].diff(days)
    Result['velocity'] = Result['STXO_diff']/Result['UTXO_Cum']
    fig9 = make_subplots(specs=[[{"secondary_y": True}]])
    fig9.add_trace(go.Scatter(x=Result['date'], y=Result['velocity'], mode='lines', name='Token Velocity'), secondary_y=False )
    fig9.add_trace(go.Scatter(x=price['Date'], y=price['PriceUSD'], mode='lines', name='Price'), secondary_y=True )
    fig9.update_layout(xaxis_title='Date', yaxis_title='Velocity', title = 'Token Velocity and Price for '+self.currency)
    fig9.update_yaxes(type="log", title_text ="Price (USD)", secondary_y=True)
    fig9.update_yaxes(tickvals=[0.1, 1, 10, 100, 1000, 10000], secondary_y=True)
    fig9.update_layout(legend=dict(
          orientation="h",
          yanchor="bottom",
          y=1.02,
          xanchor="right",
          x=0.95
      ))
    py.iplot(fig9)
    return fig9


## Call the instance 

In [ ]:
Altcoin = Altcoin(URL, currency="dash")

## Figure 1: Dist of Accumulated UTXO and STXO

In [ ]:
Dist = Altcoin.Dist()

## Figure 2:  Dist of Accumulated/Daily UTXO and STXO respectively

### Figure 2.1 UTXO

In [ ]:
Dist_UTXO = Altcoin.Dist_UTXO()

###Figure 2.2: UTXO (NUM)

In [ ]:
Dist_UTXO_numb = Altcoin.Dist_UTXO_num()

### Figure 2.3 Accumulated STXO

In [ ]:
Dist_STXO =Altcoin.Dist_STXO()

### Figure 2.4 Accumulated STXO

In [ ]:
Dist_STXO_num = Altcoin.Dist_STXO_num()

### Figure 2.5 Daily STXO

In [ ]:
Dist_daily_STXO = Altcoin.Dist_daiy_STXO()

###Figure 2.6 WAL

In [ ]:
WAL = Altcoin.WAL()

## Figure 3: Other Variables of Economic Value

### Figure 3.1: Supply

In [ ]:
Supply = Altcoin.Supply()

In [ ]:
days=60
Velocity =Altcoin.Velocity(Result, days)

In [ ]:
days=30
Velocity =Altcoin.Velocity(Result, days)